In [0]:
dbutils.widgets.removeAll()

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import functions as F

In [0]:
dbutils.widgets.text("catalogo", "catalog_jptq")
dbutils.widgets.text("esquema_stage", "silver_jptq")
dbutils.widgets.text("esquema_target", "golden_jptq")
dbutils.widgets.text("tabla", "TBL_DIM_PELICULA_T1")
dbutils.widgets.text("storageLocation", "abfss://unity-catalog-jptq@adlsjptq0126.dfs.core.windows.net")

In [0]:
catalogo = dbutils.widgets.get("catalogo")
esquema_stage = dbutils.widgets.get("esquema_stage")
esquema_target = dbutils.widgets.get("esquema_target")
tabla = dbutils.widgets.get("tabla")
storageLocation = dbutils.widgets.get("storageLocation")

In [0]:
def fn_create_table_dim_pelicula(catalogo: str, esquema: str, tabla: str):
   if spark.catalog.tableExists(f"{catalogo}.{esquema_target}.{tabla}"):
      print(f"La tabla ya existe: {catalogo}.{esquema_target}.{tabla}")
   else:
      spark.sql(f"""CREATE TABLE IF NOT EXISTS {catalogo}.{esquema_target}.{tabla} (
      SK_DIM_PELICULA BIGINT GENERATED ALWAYS AS IDENTITY,
      COD_PELICULA STRING, 
      DES_PELICULA STRING,
      DES_DURACION_PELICULA STRING,
      DES_TOP_ACTORES STRING,
      DES_RUTA_POSTER_FRONTAL STRING,
      DES_RUTA_POSTER_TRASERA STRING,
      FEC_LANZAMIENTO DATE,
      FEC_CARGA DATE,
      FEC_ACTUALIZACION DATE
      )
      USING DELTA
      LOCATION '{storageLocation}/golden-jptq/{tabla}'""")
      print(f"Tabla creada correctamente: {catalogo}.{esquema_target}.{tabla}")

In [0]:
fn_create_table_dim_pelicula(catalogo,esquema_target,tabla)

In [0]:
def fn_insert_table_dim_pelicula(catalogo: str, esquema: str, tabla: str):
    spark.sql(f"""MERGE INTO {catalogo}.{esquema_target}.{tabla} AS TARGET
    USING {catalogo}.{esquema_stage}.TBL_TMP_PELICULA AS SOURCE
    ON TARGET.COD_PELICULA = SOURCE.ID_PELICULA
    WHEN MATCHED THEN
    UPDATE SET
    TARGET.DES_PELICULA = SOURCE.DES_PELICULA,
    TARGET.DES_DURACION_PELICULA = SOURCE.DES_DURACION_PELICULA,
    TARGET.DES_TOP_ACTORES = SOURCE.DES_TOP_ACTORES,
    TARGET.DES_RUTA_POSTER_FRONTAL = SOURCE.DES_RUTA_POSTER_FRONTAL,
    TARGET.DES_RUTA_POSTER_TRASERA = SOURCE.DES_RUTA_POSTER_TRASERA,
    TARGET.FEC_LANZAMIENTO = SOURCE.FEC_LANZAMIENTO,
    TARGET.FEC_ACTUALIZACION = CURRENT_DATE()
    WHEN NOT MATCHED THEN
    INSERT (
    COD_PELICULA, 
    DES_PELICULA,
    DES_DURACION_PELICULA,
    DES_TOP_ACTORES,
    DES_RUTA_POSTER_FRONTAL,
    DES_RUTA_POSTER_TRASERA,
    FEC_LANZAMIENTO,
    FEC_CARGA,
    FEC_ACTUALIZACION
    )
    VALUES (
    SOURCE.ID_PELICULA,    
    SOURCE.DES_PELICULA,
    SOURCE.DES_DURACION_PELICULA,
    SOURCE.DES_TOP_ACTORES,
    SOURCE.DES_RUTA_POSTER_FRONTAL,
    SOURCE.DES_RUTA_POSTER_TRASERA,
    SOURCE.FEC_LANZAMIENTO,
    CURRENT_DATE(),
    TO_DATE('31/12/9999','dd/MM/yyyy')
    )""")
    print(f"Insersion Correcta en {catalogo}.{esquema_target}.{tabla}")

In [0]:
fn_insert_table_dim_pelicula(catalogo,esquema_target,tabla)

In [0]:
%sql
SELECT
*
FROM
catalog_jptq.golden_jptq.tbl_dim_pelicula_t1